In [3]:
from src.scripts.graph_analysis import connect_tg, calculate_page_rank
from src.utils.constants import PAGE_RANK_PARAMS,E_TYPE
import pandas as pd
from tqdm import tqdm
from web3 import Web3
import json
from src.utils.constants import INFURA_API_KEY, INFURA_URL, ETHERSCAN_API_KEY
w3 = Web3(Web3.HTTPProvider(INFURA_URL + INFURA_API_KEY))
from src.scripts.graph_analysis import calculate_personalized_page_rank

In [28]:
conn =connect_tg()

Connecting To TigerGraph...


In [42]:
result = conn.runInstalledQuery(
        queryName="recommendAddresses",
        params={"source": '0xB0Cf943Cf94E7B6A2657D15af41c5E06c2BFEA3D', "topK": 10},
    )
df = pd.DataFrame(
    [
        {
            "from": d.get("from_id"),
            "to": d.get("to_id"),
            "timestamp": d.get('attributes').get('timestamp')[0],
            "value": d.get('attributes').get('value')[0],
            "gas": d.get('attributes').get('gas')[0],
            "gasPrice": d.get('attributes').get('gasPrice')[0],
            "blockNumber": d.get('attributes').get('blockNumber')[0],
        }
        for d in result[0].get("@@edgeList")
    ]
)

In [4]:
result = conn.runInstalledQuery(
        queryName="recommendAddresses", params={"source": '0xA090e606E30bD747d4E6245a1517EbE430F0057e', "topK": 10},
    )


In [ ]:
pd.DataFrame([{'Address': d.get('v_id'), 'Score': d.get('attributes').get('recommendations.@rank')} for d in result[0].get("recommendations")])

In [5]:
from ens import ENS
ns = ENS.fromWeb3(w3)

In [6]:
domain = ns.name('0xE7c7652969aB78b74c7921041416A82632EA7b2d')

In [19]:
df = pd.read_csv('data/financial_transactions.csv')

In [1]:
import networkx as nx
from node2vec import Node2Vec
import pandas as pd

In [4]:
f_transactions = pd.read_csv("data/financial_transactions.csv")


In [5]:
f_transactions

,_id,blockNumber,sender,receiver,value,timestamp,gas,gasPrice
0,623cc8b73b3a8d2805cd14ad,14450902,0x91ADf14f4C0782634E04Dfc6e9Be16d950AA4daA,0x316782400aC34966796C58f1Bb46a0ee8a07d234,2.638664,1648150525,31000,71000000000
1,623cc8b73b3a8d2805cd14af,14450902,0x8216874887415e2650D12D53Ff53516F04a74FD7,0x5ead7113806FBE599B75f9b1D9e77A2c7A2F0bF2,0.011200,1648150525,21000,55246208935
2,623cc8b73b3a8d2805cd14b3,14450902,0x3cD751E6b0078Be393132286c442345e5DC49699,0xA40d625fa65ea444FC1FDF3c2231b4b38dF5190c,0.096204,1648150525,21000,54746208935
3,623cc8b73b3a8d2805cd14b5,14450902,0xb5d85CBf7cB3EE0D56b3bB207D5Fc4B82f43F511,0x128fd52797A2df3d4E4c66F7B4BCfc41b9E9F2F4,0.313482,1648150525,21000,54746208935
4,623cc8b73b3a8d2805cd14b6,14450902,0xeB2629a2734e272Bcc07BDA959863f316F4bD4Cf,0x03E1729a4a74FCF64F3747113ef1Ad9962fc4129,0.016580,1648150525,21000,54746208935
...,...,...,...,...,...,...,...,...
226242,6241f10fc80a5ab9df45907f,14474915,0x8C212bF75cdfeA6A4C5e40E7023f80eDB17f80F4,0x7f268357A8c2552623316e2562D90e642bB538E5,0.158000,1648473963,258274,48057120735
226243,6241f10fc80a5ab9df459080,14474915,0x216648069CC4cb9072cACc9289c611df1F95a7b2,0x37C37d9c626d192676664785356faC3fA4ca0AC3,0.005960,1648473963,21000,48057120735
226244,6241f10fc80a5ab9df459082,14474915,0xa2Ab4a421e5CB267939840Cb804b25fbfA08eD3e,0xA11C5fAaF4Dc6E559E039aDf4422e1C9Cfd59998,0.007026,1648473963,21000,48057120735
226245,6241f10fc80a5ab9df459084,14474915,0xf843B85a9AbC591624b308F56cc72d7402FfcF9C,0xdD366E22d2EE4bBfDbd9E85457FafD8F8C3E64c4,0.050000,1648473963,21000,48057120735


In [16]:
graph = nx.from_pandas_edgelist(f_transactions, source='sender',target='receiver', edge_attr='value')


In [17]:
node2vec = Node2Vec(graph,weight_key='value')


Generating walks (CPU: 1): 100%|██████████| 10/10 [2:14:05<00:00, 804.58s/it]


In [18]:
model = node2vec.fit(window=10, min_count=1, batch_words=4) 

In [49]:
model.wv.most_similar('0xc0E013dBE8D1C3e07deB85725D02AafEe3191EcF')

[('0xF9175FBF88Ab9AD0b4E85EF21d3d16759BE82294', 0.9599786996841431),
 ('0x5bC81870F8F5F955E40871853c84bf06cd79A3Bb', 0.9549767374992371),
 ('0x5979dBdE10E792a2b8D570D766c893b55C2B84eb', 0.9546607136726379),
 ('0x99Eb42ef862173317da23D0A6B940fd65E3851E6', 0.9518017172813416),
 ('0x0167759648Bdf7dEa7C5c143205522eABB149A56', 0.9500256776809692),
 ('0x345Acc938b082155971bE320B18bff31072963D0', 0.9493658542633057),
 ('0x3Ba0615bb359d37066612915F043089Aa7019928', 0.9455761313438416),
 ('0xd19c00c28D4434eB8ec58B4c1AeA5DADA6B0aE31', 0.9448641538619995),
 ('0x648279F5A2e4190c2EeC70733297035F5267b6F5', 0.9420770406723022),
 ('0xb5D4711dA6324bF904C9FbaABc2F8Cd377845934', 0.9419685006141663)]

In [26]:
EMBEDDING_FILENAME = 'web3.embeddings'
EMBEDDING_MODEL_FILENAME = 'web3.model'

In [33]:
model.wv.save_word2vec_format(EMBEDDING_FILENAME)
model.save(EMBEDDING_MODEL_FILENAME)

In [22]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [25]:
model = Word2Vec.load(EMBEDDING_MODEL_FILENAME)
embeddings = KeyedVectors.load_word2vec_format(EMBEDDING_FILENAME)


In [27]:
model.wv.save_word2vec_format(EMBEDDING_FILENAME)
model.save(EMBEDDING_MODEL_FILENAME)